In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
import win32com.client as win32

tabela_produtos = pd.read_excel('tabela.xlsx')
navegador = webdriver.Chrome()

def lista_termos(produto, termos_banidos):
    produto = produto.lower()
    lista_termo_produto = produto.split(' ')
    
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    
    return produto, lista_termo_produto, lista_termos_banidos


def verificacao_termos(lista_termo_produto, lista_termos_banidos, nome):
    tem_todos_termos_produto = True
    for termo in lista_termo_produto:
        if termo not in nome:
            tem_todos_termos_produto = False
            

    tem_termos_banidos = False
    for banidos in lista_termos_banidos:
        if banidos in nome:
            tem_termos_banidos = True
          
    return tem_todos_termos_produto, tem_termos_banidos


def restricao_precos(preco_minimo, preco_maximo):
    minimo = float(preco_minimo)
    maximo = float(preco_maximo)
    
    return minimo, maximo


def buscar_google_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    navegador.get('https://www.google.com/')
    #funcao da lista de termos OBJ: o produto nao é obrigatorio ser retornado, é somente para fica padrozinado em letras minusculos
    produto, lista_termo_produto, lista_termos_banidos = lista_termos(produto, termos_banidos)    
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(produto, Keys.ENTER)
    
    #funcao restricao de precos
    preco_minimo, preco_maximo=restricao_precos(preco_minimo, preco_maximo)
    
    pesquisa = navegador.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    
    

    for p in pesquisa:
        if 'Shopping' in p.text:
            p.click()
            break
    resultados = navegador.find_elements(By.CLASS_NAME, 'i0X6df')

    lista_iphones_12 = []

    for resultado in resultados:
        try:
            nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
            nome = nome.lower()
            #funcao verificicao de termos
            tem_todos_termos_produto, tem_termos_banidos = verificacao_termos(lista_termo_produto, lista_termos_banidos, nome)
        
            if not tem_termos_banidos and tem_todos_termos_produto: 
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(",", "").replace(" + tax", "")
                preco = float(preco)
                
                if preco_minimo <= preco <= preco_maximo:
                    lojas = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    pai = lojas.find_element(By.XPATH, '..')
                    link = pai.get_attribute('href')

                    lista_iphones_12.append((nome, preco, link))
        except:
            pass
                    
    return lista_iphones_12


def buscar_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    navegador.get('https://www.buscape.com.br/')
    lista_notebook = []
    #funcao da lista de termos OBJ: o produto nao é obrigatorio ser retornado, é somente para fica padrozinado em letras minusculos
    produto, lista_termo_produto, lista_termos_banidos = lista_termos(produto, termos_banidos)
    #funcao restricao de precos
    preco_minimo, preco_maximo=restricao_precos(preco_minimo, preco_maximo)
    
    navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__FInnF').send_keys(produto, Keys.ENTER)
    time.sleep(3)

    resultados = navegador.find_elements(By.CLASS_NAME, 'Cell_Content__1630r')
    time.sleep(3)
    for resultado in resultados:
        try:
            nome = resultado.find_element(By.CLASS_NAME, 'Text_Text___RzD-').text
            nome = nome.lower()
            #funcao verificacao de termos
            tem_todos_termos_produto, tem_termos_banidos = verificacao_termos(lista_termo_produto, lista_termos_banidos, nome)
        
            if not tem_termos_banidos and tem_todos_termos_produto:           
                preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__3s0iP').text
                preco = preco.replace("R$ ", "").replace(".","").replace(",",".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:        
                    pai = resultado.find_element(By.CLASS_NAME, 'Cell_Body__MIfCb')
                    filho = pai.find_element(By.XPATH, "..")
                    link = filho.get_attribute("href")
                    lista_notebook.append((nome, preco, link))
        except:
            pass


    return lista_notebook


def buscar_zoom_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    navegador.get("https://www.zoom.com.br/?")
    #funcao da lista de termos OBJ: o produto nao é obrigatorio ser retornado, é somente para fica padrozinado em letras minusculos
    produto, lista_termo_produto, lista_termos_banidos = lista_termos(produto, termos_banidos)
    
    #funcao restricao de precos
    preco_minimo, preco_maximo=restricao_precos(preco_minimo, preco_maximo)

    lista_placa_video = []


    
    navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__FInnF').send_keys(produto, Keys.ENTER)
    time.sleep(3)
    resultados = navegador.find_elements(By.CLASS_NAME, 'Cell_Cell__1YAxR')
    time.sleep(3)
    for resultado in resultados:
        try:
            nome = resultado.find_element(By.CLASS_NAME, "Cell_Name__jnsS-").text
            nome = nome.lower()
            #funcao verificao de termos
            tem_todos_termos_produto, tem_termos_banidos = verificacao_termos(lista_termo_produto, lista_termos_banidos, nome)

            if not tem_termos_banidos and tem_todos_termos_produto:           
                preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__3s0iP').text
                preco = preco.replace("R$ ", "").replace(".", "").replace(",", ".")
                preco = float(preco)       
                if preco_minimo <= preco <= preco_maximo:         
                    link = resultado.find_element(By.CLASS_NAME, 'Cell_Body__MIfCb')
                    link = link.find_element(By.XPATH, '..')
                    link = link.get_attribute('href')
                    lista_placa_video.append((nome, preco, link))
        except:
            pass
    return lista_placa_video


def criando_Tabela_df(tabela_produtos):
    
    tabela_preco_df = pd.DataFrame()

    for linha in tabela_produtos.index:
        produto = tabela_produtos.loc[linha, 'Nome Produto']
        termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
        preco_minimo =  tabela_produtos.loc[linha, 'Preço mínimo']
        preco_maximo = tabela_produtos.loc[linha, 'Preço máximo']
        google_shopping = buscar_google_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo)

        if google_shopping:
            tabela_google_shopping = pd.DataFrame(google_shopping, columns=["Produto", "Preco", "Link"])
            tabela_preco_df = tabela_preco_df.append(tabela_google_shopping)
        else:
            google_shopping = None


        buscape = buscar_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)

        if buscape:
            tabela_buscape_shopping = pd.DataFrame(buscape, columns=["Produto", "Preco", "Link"])
            tabela_preco_df = tabela_preco_df.append(tabela_buscape_shopping)

        else:
            buscape = None


        zoom = buscar_zoom_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo)

        if zoom:
            tabela_zoom_shopping = pd.DataFrame(zoom, columns=["Produto", "Preco", "Link"])
            tabela_preco_df = tabela_preco_df.append(tabela_zoom_shopping)
        else:
            zoom = None
    
    return tabela_preco_df


def enviando_email(win32, tabela_produtos):
    #criando um dataframe
    tabela_df = criando_Tabela_df(tabela_produtos)
    if len(tabela_df.index) > 0:
        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        mail.To = 'destinatario@gmail.com'
        mail.Subject = 'Produto(s) Encontrado(s) na faixa de preco desejada'
        mail.HTMLBody =f"""
        <p><strong> Prezado,</strong></p>
        <p> Encontrados alguns produtos em oferta dentro da faixa de preço desejado, segue a tabela com detalhes </p>
        {tabela_df.to_html(index=False)}
        <p>Qualquer dúvida estou á disposicao </p>
        <p> att., </p>

        """
        mail.Send()
    return tabela_df


#enviando email
enviando_email(win32, tabela_produtos)


#para criar uma tabela excel descomente as linhas abaixo
#tabela_preco_df = tabela_preco_df.reset_index(drop=True)
#tabela_preco_df.to_excel("lista_precos.xlsx", index=False)
navegador.quit()
print("E-mail enviado!")